In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2003','era_2003.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2003-01-01 ... 2003-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T22:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2003-01-01 ... 2003-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T22:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2003-01-01 00:00:00 1000.0         49.25    -124.85    95.699524  0.004736
                                            -124.60    92.176086  0.004641
                                            -124.35    87.871399  0.004652
                                            -124.10    86.851868  0.004756
                                            -123.85    89.351868  0.004960
...                                                          ...       ...
2003-12-31 21:00:00 1000.0         24.50    -68.10     61.919025  0.008621
                                            -67.85     62.094807  0.008630
                                            -67.60     62.100666  0.008649
                                            -67.35     62.305744  0.008676
                                            -67.10     62.706135  0.008683

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2003-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T22:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2003-12-31 1000.0         49.25    -124.85    82.328117  0.006166
                                   -124.60    79.443169  0.006148
                                   -124.35    77.157928  0.006048
                                   -124.10    74.857422  0.005969
                                   -123.85    71.801666  0.005989
...                                                 ...       ...
                          24.50    -68.10     79.821350  0.014878
                                   -67.85     79.808891  0.014870
                                   -67.60     79.764198  0.014859
                                   -67.35     79.676613  0.014845
                                   -67.10     79.581009  0.014833

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2003-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 82.33 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006166 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T22:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,geometry
0,19,175,00465276,19175,Union,Union County,06,H1,G4020,None,None,None,A,"POLYGON ((-94.47051 40.97504, -94.47050 40.975..."
1,19,177,00465277,19177,Van Buren,Van Buren County,06,H1,G4020,None,None,None,A,"POLYGON ((-91.96059 40.90070, -91.95925 40.900..."
2,20,097,00485013,20097,Kiowa,Kiowa County,06,H1,G4020,None,None,None,A,"POLYGON ((-99.01471 37.67895, -99.01473 37.678..."
3,20,109,00485019,20109,Logan,Logan County,06,H1,G4020,None,None,None,A,"POLYGON ((-100.89562 39.13329, -100.89404 39.1..."
4,20,003,00484971,20003,Anderson,Anderson County,06,H1,G4020,None,None,None,A,"POLYGON ((-95.51741 38.20619, -95.51741 38.206..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,35,015,00936829,35015,Eddy,Eddy County,06,H1,G4020,None,16100,None,A,"POLYGON ((-104.09554 32.00000, -104.09586 32.0..."
3229,40,015,01101795,40015,Caddo,Caddo County,06,H1,G4020,None,None,None,A,"POLYGON ((-98.36984 35.55133, -98.36964 35.551..."
3230,40,025,01101800,40025,Cimarron,Cimarron County,06,H1,G4020,None,None,None,A,"POLYGON ((-102.14252 36.50032, -102.14380 36.5..."
3231,40,069,01101822,40069,Johnston,Johnston County,06,H1,G4020,None,None,None,A,"POLYGON ((-96.74415 34.17223, -96.74416 34.172..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-94.47051 40.97504, -94.47050 40.975...
1       POLYGON ((-91.96059 40.90070, -91.95925 40.900...
2       POLYGON ((-99.01471 37.67895, -99.01473 37.678...
3       POLYGON ((-100.89562 39.13329, -100.89404 39.1...
4       POLYGON ((-95.51741 38.20619, -95.51741 38.206...
                              ...                        
3228    POLYGON ((-104.09554 32.00000, -104.09586 32.0...
3229    POLYGON ((-98.36984 35.55133, -98.36964 35.551...
3230    POLYGON ((-102.14252 36.50032, -102.14380 36.5...
3231    POLYGON ((-96.74415 34.17223, -96.74416 34.172...
3232    POLYGON ((-84.27514 39.28921, -84.27467 39.289...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0    -94.470506  40.975043
     1    -94.470501  40.975551
     2    -94.470512  40.978164
     3    -94.470498  40.978804
     4    -94.470502  40.979333
...              ...        ...
3232 3894 -84.275637  39.289242
     3895 -84.275550  39.289236
     3896 -84.275516  39.289234
     3897 -84.275508  39.289233
     3898 -84.275143  39.289207

[7781581 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3233 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 27s
Wall time: 6min 36s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,67.176459,0.006445
1,66.585112,0.006598
2,60.999215,0.006994
3,56.733848,0.006096
4,65.645063,0.00749
...,...,...
3228,41.512158,0.006048
3229,60.542624,0.007727
3230,49.376911,0.005749
3231,64.208718,0.008979


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
CNTYIDFP      object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CSAFP         object
CBSAFP        object
METDIVFP      object
FUNCSTAT      object
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD',
                              'LSAD','CLASSFP','MTFCC','CSAFP','CBSAFP','METDIVFP',
                             'FUNCSTAT'], axis=1)
county_gdf

,NAME,geometry,fips
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003
...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,67.176459,0.006445
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,66.585112,0.006598
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,60.999215,0.006994
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,56.733848,0.006096
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,65.645063,0.00749
...,...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,41.512158,0.006048
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,60.542624,0.007727
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,49.376911,0.005749
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,64.208718,0.008979


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,67.176459,0.006445
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,66.585112,0.006598
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,60.999215,0.006994
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,56.733848,0.006096
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,65.645063,0.00749
...,...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,41.512158,0.006048
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,60.542624,0.007727
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,49.376911,0.005749
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,64.208718,0.008979


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Union,19175,67.176458815540755,0.006445407425240
1,Van Buren,19177,66.585111540879609,0.006598232989055
2,Kiowa,20097,60.999214739902655,0.006994033023982
3,Logan,20109,56.733847855139963,0.006095586756198
4,Anderson,20003,65.645062714374774,0.007490220196843
...,...,...,...,...
3228,Eddy,35015,41.512158415066459,0.006047902689259
3229,Caddo,40015,60.542624430963471,0.007727062766558
3230,Cimarron,40025,49.376910665815750,0.005748684763569
3231,Johnston,40069,64.208717833903179,0.008979032719598


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
1536,Autauga,1001,71.152047054515833,0.010074853188795
441,Baldwin,1003,74.399172844073590,0.011615945151359
2221,Barbour,1005,71.229239958483802,0.010308249362289
2388,Bibb,1007,72.651517962262062,0.009904532979751
2153,Blount,1009,73.001927053727783,0.009520829871887
...,...,...,...,...
1193,Sweetwater,56037,55.617026320013984,0.003576894567435
1865,Teton,56039,64.274432029956586,0.003292078917149
2348,Uinta,56041,59.220463661859569,0.003552831216642
2349,Washakie,56043,54.687323202686905,0.004052235973441


In [21]:
county_var.to_pickle('era_2003.pkl')